In [1]:
from __future__ import print_function

from IPython.display import display
import pandas as pd
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Dashboard COVID 19 Voilà

In [2]:
with open(file='C:/Users/AxelArcidiaco/Documents/GRETA/Projet Dev/Business Intelligence/Story 5/workshops/workshops/voila_workshop/red-and-sky-blue.png', mode='rb') as f_image :
    image = f_image.read()
    display(widgets.Image(value=image, format='png'))

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/AxelArcidiaco/Documents/GRETA/Projet Dev/Business Intelligence/Story 5/workshops/workshops/voila_workshop/red-and-sky-blue.png'

In [ ]:
dfs = dict()
dfs["confirmed"] = pd.read_csv('../covid_19/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
dfs['death'] = pd.read_csv('../covid_19/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
dfs['recovered'] = pd.read_csv('../covid_19/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
dfs['country'] = pd.read_csv('../covid_19/csse_covid_19_time_series/UID_ISO_FIPS_LookUp_Table.csv')

In [ ]:
for df in dfs:
    dfs[df].columns = map(str.lower, dfs[df].columns)


In [ ]:
for df in dfs:
    dfs[df] = dfs[df].rename(columns = {'province/state' : 'state', 'country/region' : 'country'})

In [ ]:
dfs['country'].rename(columns = {'province_state' : 'state', 'country_region' : 'country'}, inplace=True)

In [ ]:
dfs['country'].drop(columns=['uid', 'iso2', 'iso3', 'code3', 'fips', 'admin2', 'combined_key'], inplace=True)

In [ ]:
#dfs["confirmed"].head()

In [ ]:
for df in dfs:
    dfs[df]['state'].fillna("", inplace=True)

In [ ]:
#dfs["death"]

In [ ]:
for df in ['confirmed', 'death', 'recovered']:
    dfs[df][df] = dfs[df].loc[:,'1/22/20':'12/14/22'].sum(axis=1)

In [ ]:
#dfs["death"]

In [ ]:
#dfs["confirmed"].set_index(['country', 'state'], inplace=True)

In [ ]:
#for df in ['confirmed', 'death', 'recovered']:
#    dfs[df].set_index(['country', 'state'], inplace=True)

In [ ]:
for df in ['confirmed', 'death', 'recovered']:
    dfs['country'] = dfs['country'].join(other=dfs[df].set_index(['country', 'state'])[df], on=['country', 'state'], how='inner')

In [ ]:
#dfs["country"] = dfs["country"].join(other=dfs["confirmed"]['confirmed'], on=['country', 'state'], how='inner')

In [ ]:
#dfs["country"].head()

In [ ]:
dfs['country_sorted'] = dfs['country'].sort_values('confirmed', ascending=False)

In [ ]:
#dfs['country_sorted'].head(5)

# Statistiques par pays
Nombres de de cas confirmés, de morts et de cas soignés par pays classé par nombre de cas confirmés.


In [ ]:
def highlight_col(df):
    red = 'background-color : #F5B7B1'
    purple = 'background-color : #D2B4DE'
    grey = 'background-color : #E5E8E8'
    temp_df = pd.DataFrame('', index=df.index, columns=df.columns)
    temp_df.loc[:, 'confirmed'] = purple
    temp_df.loc[:, 'death'] = red
    temp_df.loc[:, 'recovered'] = grey
    return temp_df

def display_country(nb : str=0) :
    if not nb.isdigit() :
        nb = 0
    nb = int(nb)
    if nb == 0 :
        print("Entrez une valeur dans le champ nombre")
    else :
        display(dfs['country_sorted'][['country', 'state', 'population', 'confirmed', 'death', 'recovered']].head(nb).style.apply(highlight_col, axis=None).hide(axis='index'))

stats_pays = interactive(display_country, nb=widgets.Text(value="10", description='Nombre de pays :'))
stats_pays

interactive(children=(Text(value='10', description='Nombre de pays :'), Output()), _dom_classes=('widget-inter…

In [ ]:
#dfs['country_sorted'][['country', 'state', 'population', 'confirmed', 'death', 'recovered']].head(5).style.apply(highlight_col, axis=None).hide(axis='index')

In [ ]:
import plotly.express as plot_exp

In [ ]:
def graph_pays_confirmed(nb) :
    fig = plot_exp.scatter(dfs['country_sorted'].head(nb), x='country', y='confirmed', size='confirmed', color='country', hover_name='country', size_max=60)
    return fig.show()

graph = interactive(graph_pays_confirmed, nb=widgets.IntSlider(value=10, min='1', max=20, step=1, description='Nombre de pays :', continuous_update=False))
graph

interactive(children=(IntSlider(value=10, continuous_update=False, description='Nombre de pays :', max=20, min…

## Courbes des cas confirmés et des morts par pays

Rentrez un nom de pays et appuyer sur le bouton 'run interract'

In [ ]:
import plotly.graph_objects as plot_graph

def plot_cases_for_country(country : str) :
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6,8]
    line_size = [4,5]
    country = country.capitalize()

    df_list = [dfs['confirmed'], dfs['death']]

    fig = plot_graph.Figure()

    for i, df in enumerate(df_list):
        if country.lower() == 'world' :
            x_data = np.array(list(df.loc[:,'1/22/20':'12/14/22'].columns))
            y_data = np.sum(np.asarray(df.loc[:,'1/22/20':'12/14/22']), axis=0)
        else :
            x_data = np.array(list(df.loc[:,'1/22/20':'12/14/22'].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].loc[:,'1/22/20':'12/14/22']), axis=0)

        fig.add_trace(plot_graph.Scatter(x=x_data, y=y_data, mode='lines+markers', 
                                        name=labels[i],
                                        line=dict(color=colors[i], width=line_size[i]), 
                                        connectgaps=True, 
                                        text = "Total " + str(labels[i]) + ": "+ str(y_data[-1])
                                        )
                        )

    fig.show()

#plot_cases_for_country('France')

graph_confirmed_death = interactive(plot_cases_for_country, {'manual': True}, country=widgets.Text(value='World', description='Pays :'))
graph_confirmed_death

interactive(children=(Text(value='World', description='Pays :'), Button(description='Run Interact', style=Butt…

<hr>

In [ ]:
plot_exp.bar(dfs['country_sorted'].head(10), x='country', y='confirmed', title='Les 10 pays les plus touchés en cas détectés')

In [ ]:
plot_exp.bar(dfs['country'].sort_values('death', ascending=False).head(10), x='country', y='death', title='Les 10 pays les plus touchés en nombre de morts')

In [ ]:
plot_exp.bar(dfs['country'].sort_values('recovered', ascending=False).head(10), x='country', y='recovered', title='Les 10 pays ayant eut le plus de cas guéris')

In [ ]:
import folium

## Carte mondiale des cas confirmés de COVID 19

In [ ]:
world_map = folium.Map(location=[11,0], tiles='cartodbpositron', zoom_start=2, max_zoom=6, min_zoom=2)

df_confirmed = dfs['confirmed'].dropna()

for i in range(len(df_confirmed)):
    folium.Circle(
        location=[df_confirmed.iloc[i]['lat'], df_confirmed.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(df_confirmed.iloc[i, -1] + 1.00001))) + 0.2) * 15000,
        fill_color = 'blue',
        color='red',
        tooltip="<div style='margin:0; background-color:black; color:white;'>"+
                "<h4 style='text-align:center;front-weight: bold'>"+df_confirmed.iloc[i]['country']+"</h4>"+
                "<hr style='margin:10px; color:white;'>"+
                "<ul style='color:white; list-style-type:circle; align-item:left;padding-left:20px;padding-right:20px'>"+
                    "<li>Confirmed: "+str(df_confirmed.iloc[i,-1])+"</li>"+
                    "<li>Deaths:   "+str(dfs['death'].dropna().iloc[i, -1])+"</li>"+
                    "<li>Death rate:    "+str(np.round(dfs['death'].dropna().iloc[i, -1]/(df_confirmed.iloc[i,-1]+1.00001)*100,2))+"</li>"+
                "</ul></div>",
    ).add_to(world_map)

world_map